In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import glob
import json

#  Function to plot interactions with walls and non-self objects.
from scipy.stats import stats
from scipy.stats import sem

import statistics

def plot_all_interactions(game_types, agent_types):
    for game in game_types:
        plot_interaction(game, agent_types)

def plot_interaction(game_type, agent_types):
    w_count = []
    ns_count = []
    w_sem_all = []
    ns_sem_all = []
    for agent in agent_types:
        w, ns, w_sem, ns_sem = get_interaction_counts(game_type, agent)
        w_count.append(w)
        ns_count.append(ns)
        w_sem_all.append(w_sem)
        ns_sem_all.append(ns_sem)

    # Normalize the counts
    normalize(w_count, ns_count, w_sem_all, ns_sem_all)
    label_dict = { 'self_class': 'Self Class', 'dqn_training': 'DQN',
                       'random': 'Random', 'a2c_training': "A2C", 'trpo_training': 'TRPO', 'ppo2_training':'PPO2',
               'acer_training': 'ACER',
                    'human': 'Human'}
    agent_labels = label_dict.values()
    labels = agent_labels
    x = np.arange(len(agent_types))  # the label locations
    width = 0.2  # the width of the bars

    fig, ax = plt.subplots()
    plt.xticks(rotation=90)
    wall = ax.bar(x - width/2, w_count, width,  capsize=3, label='Wall', color="#00fa9a", yerr=w_sem_all)
    non_self = ax.bar(x + width/2, ns_count, width, capsize=3, label='Non-self', color="#e500fa", yerr=ns_sem_all)

    # Add some text for labels, title and custom x-axis tick labels, etc.
    game_titles = {
        "shuffleKeys_game": "Shuffle Keys Game",
        "contingency_game": "Contingency Game",
        "contingency_game_r0": "Contingency Game (Agent Placement is Constant Among Each Level)",
        "contingency_game_shuffled": "Shuffled Keys Game \n(Shuffled Once in Every 200 Levels)",
        "logic_game": "Logic Game",
        "logic_extended_game": "Ext Logic",
        "change_agent_game": "Agent Change Game"
    }

    ax.set_ylabel('Interaction Count (Normalized)', fontsize=10)
    ax.set_title( "Interaction Count: " + game_titles[game_type], fontweight='bold', fontsize=15 )
    ax.set_xticks(x)
    ax.set_xticklabels(labels, fontsize=10)
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    fig.tight_layout()
    plt.tight_layout()
    
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    fname = game_type + "_interactions.pdf"
    fig.savefig(fname, format='pdf')

    plt.show()

def get_interaction_counts(game_type, agent_type):
    files = glob.glob("../data/" + game_type + "/" + agent_type + "/*.json")

    if len(files) == 0:
        files = glob.glob("../data/" + game_type + "/" + agent_type + "/*/*.json") + glob.glob("../data/" + game_type + "/" + agent_type + "/*.json")
        
    w_count = 0
    ns_count = 0

    seed = 0
    curr_file_count = 0
    file_amt = len(files)
    all_seeds_w = []
    all_seeds_ns = []
    for i, file in enumerate(sorted(files, key=os.path.getmtime)):
        data = json.load(open(file))
        data = data.get("data")
        
        wall_interactions = data["wall_interactions"]
        ns_interactions = data["ns_interactions"]

        w_count += sum(wall_interactions) / len(wall_interactions)
        ns_count += sum(ns_interactions) / len(wall_interactions)

        curr_file_count += 1
        if agent_type != 'human' and curr_file_count == 20:
            all_seeds_w.append(w_count)
            all_seeds_ns.append(ns_count)
            curr_file_count = 0
            seed += 1
            w_count = 0
            ns_count = 0

        if agent_type == 'human':
            all_seeds_w.append(w_count)
            all_seeds_ns.append(ns_count)

            if (curr_file_count == file_amt) or (file_amt == 1):
                break

    seed_sem_w = sem(all_seeds_w)
    seed_sem_ns = sem(all_seeds_ns)

    seed_avg_w = statistics.mean(all_seeds_w)
    seed_avg_ns = statistics.mean(all_seeds_ns)


    return seed_avg_w, seed_avg_ns, seed_sem_w, seed_sem_ns

# normalize wall and non-self counts along with the sem's
def normalize(w, ns, w_sem, ns_sem):
    for i in range(len(w)):
        w_c = w[i]
        ns_c = ns[i]
        w_sem_c = w_sem[i]
        ns_sem_c = ns_sem[i]
        sum_c = w_c + ns_c
        if sum_c == 0:
            continue
        w[i] = w_c / sum_c
        ns[i] = ns_c / sum_c

        w_sem[i] = w_sem_c / sum_c
        ns_sem[i] = ns_sem_c / sum_c
        

In [ ]:
agent_types = ["self_class", "dqn_training", "random", "a2c_training", "trpo_training", "ppo2_training",
               "acer_training", "human"]
game_types = [ "change_agent_game"] # "logic_game", "contingency_game_shuffled", "change_agent_game",

# Plot interaction frequencies for each game and agent.
plot_all_interactions(game_types, agent_types)

In [ ]:
# Function to plot bar chart for total steps.

def plot_all_total_steps(game_types, agent_types):
    for game in game_types:
        plot_total_steps(game, agent_types)

def plot_total_steps(game_type, agent_types):
    total_mean = []
    total_sem = []
    for agent in agent_types:
        mean, sem = get_mean_and_sem(game_type, agent)
        total_mean.append(mean)
        total_sem.append(sem)

    label_dict = { 'self_class': 'Self Class', 'dqn_training': 'DQN',
                       'random': 'Random', 'a2c_training': "A2C", 'trpo_training': 'TRPO', 'ppo2_training':'PPO2',
               'acer_training': 'ACER', 'human': 'Human'}
    labels = label_dict.values()
    x = np.arange(len(labels))  # the label locations
    width = 0.2  # the width of the bars

    fig, ax = plt.subplots()
    ts = ax.bar(x, total_mean, width=width, color=["#03fc9d", "#0373fc", "#d002f5", "#fa0047", "#ff8903", "#b0b81f", "#000000",
                                                    "#0000FF"], # yerr=total_sem
                ecolor='black', capsize=5, alpha=0.5)

    # Add some text for labels, title and custom x-axis tick labels, etc.
    game_titles = {
        "shuffleKeys_game": "Shuffle Keys Game",
        "contingency_game": "Contingency Game",
        "contingency_game_r0": "Contingency Game (Agent Placement is Constant Among Each Level)",
        "contingency_game_shuffled": "Shuffled Keys Game \n(Shuffled Once in Every 200 Levels)",
        "logic_game": "Logic Game",
        "logic_extended_game": "Ext Logic",
        "change_agent_game": "Agent Change Game"
    }
    ax.set_ylabel('Total Steps (Average of All Seeds)')
    ax.set_title( "Total Steps: " + game_titles[game_type], fontweight='bold', fontsize=15 )
    ax.set_xticks(x)
    ax.set_xticklabels(labels)

    def autolabel(rects):
        """Attach a text label above each bar in *rects*, displaying its height."""
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(height),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')

    #autolabel(ts)
    
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    #plt.figure(num=None, figsize=(12, 9), dpi=80, facecolor='w', edgecolor='k')
    plt.tight_layout()

    fname = game_type + "_total_steps.pdf"
    fig.savefig(fname, format='pdf')

def get_mean_and_sem(game_type, agent_type):
    files = glob.glob("../data/" + game_type + "/" + agent_type + "/*.json")

    if len(files) == 0:
        files = glob.glob("../data/" + game_type + "/" + agent_type + "/*/*.json")

    if len(files) == 0:
        return 0, 0
    seed_count = 0
    all_counts = []
    curr_file_count = 0
    for i, file in enumerate(sorted(files, key=os.path.getmtime)):
        data = json.load(open(file))
        data = data.get("data")
        seed_count += sum(data["steps"])

        curr_file_count += 1
        if curr_file_count == 20:
            all_counts.append(seed_count)
            seed_count = 0
            curr_file_count = 0

    seed_sem = sem(all_counts)
    seed_avg = statistics.mean(all_counts)
    return seed_avg, seed_sem

In [ ]:
agent_types = ["self_class", "dqn_training", "random", "a2c_training", "trpo_training", "ppo2_training",
               "acer_training"]
game_types = ["change_agent_game"]#"logic_game", "contingency_game", "change_agent_game",

# Plot interaction frequencies for each game and agent.
plot_all_total_steps(game_types, agent_types)